In [1]:
# -*- coding: utf-8 -*-
"""
Qwen2-VL 3B 视觉模型微调脚本 - 汽车保险承保专家
基于Unsloth框架进行高效微调
"""

import json
import os
from PIL import Image
from unsloth import FastVisionModel  
import torch

# 支持的4bit预量化模型列表
# fourbit_models = [
#     "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",
#     "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit", 
#     "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",
# ]

print("正在加载Qwen2.5-VL-3B模型...")
# 加载预训练模型和分词器
model, tokenizer = FastVisionModel.from_pretrained(
    "/root/autodl-tmp/models/Qwen/Qwen2.5-VL-3B-Instruct",  # 使用Qwen2.5-VL-3B模型
    #load_in_4bit = True,  # 使用4bit量化减少显存使用
    use_gradient_checkpointing = "unsloth",  # 使用梯度检查点节省显存
)

print("配置LoRA参数...")
# 配置LoRA适配器进行参数高效微调
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers = True,  # 微调视觉层
    finetune_language_layers = True,  # 微调语言层  
    finetune_attention_modules = True,  # 微调注意力模块
    finetune_mlp_modules = True,  # 微调MLP模块
    
    r = 16,  # LoRA秩，越大精度越高但可能过拟合
    lora_alpha = 16,  # LoRA alpha参数，建议等于r
    lora_dropout = 0,  # LoRA dropout
    bias = "none",  # 偏置设置
    random_state = 3407,  # 随机种子
    use_rslora = False,  # 是否使用rank stabilized LoRA
    loftq_config = None,  # LoftQ配置
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-09-03 22:10:53.819040: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-03 22:10:53.840407: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756908653.865471   52134 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756908653.873280   52134 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756908653.892974   52134 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 09-03 22:11:01 __init__.py:183] Automatically detected platform cuda.
正在加载Qwen2.5-VL-3B模型...
==((====))==  Unsloth 2025.3.19: Fast Qwen2_5_Vl patching. Transformers: 4.51.3. vLLM: 0.7.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.65 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


配置LoRA参数...


In [2]:
import pandas as pd
import os

print("加载训练数据...")
# 加载Excel格式的训练数据
def load_excel_dataset(file_path):
    """加载Excel格式的数据集"""
    try:
        df = pd.read_excel(file_path)
        print(f"Excel文件列名: {list(df.columns)}")
        print(f"数据集形状: {df.shape}")
        return df
    except Exception as e:
        print(f"读取Excel文件时出错: {e}")
        return None

# 数据转换函数 - 处理Excel格式数据
def convert_excel_to_training_format(df):
    """将Excel格式转换为训练格式，加载本地图片"""
    converted_data = []
    
    for idx, row in df.iterrows():
        # Excel格式: id, prompt, image, response
        image_path = row["image"]
        prompt = row["prompt"]
        response = row["response"]
        
        if pd.notna(image_path) and os.path.exists(image_path):
            try:
                # 加载图片
                image = Image.open(image_path).convert('RGB')
                
                # 创建训练样本
                conversation = {
                    "messages": [
                        {
                            "role": "user",
                            "content": [
                                {"type": "text", "text": prompt},
                                {"type": "image", "image": image}
                            ]
                        },
                        {
                            "role": "assistant", 
                            "content": [
                                {"type": "text", "text": response}
                            ]
                        }
                    ]
                }
                converted_data.append(conversation)
                print(f"成功处理样本 {idx + 1}: {image_path}")
            except Exception as e:
                print(f"处理图片 {image_path} 时出错: {e}")
        else:
            print(f"警告：图片文件不存在或路径为空 {image_path}")
    
    return converted_data

# 加载并转换数据集
train_df = load_excel_dataset("qwen-vl-train.xlsx")
if train_df is not None:
    converted_dataset = convert_excel_to_training_format(train_df)
else:
    print("无法加载数据集，程序退出")
    exit()

print(f"成功加载 {len(converted_dataset)} 个训练样本")

加载训练数据...
Excel文件列名: ['id', 'prompt', 'image', 'response']
数据集形状: (2, 4)
成功处理样本 1: images/1-vehicle-odometer-reading.jpg
成功处理样本 2: images/2-vehicle-odometer-reading.jpg
成功加载 2 个训练样本


In [3]:
converted_dataset

[{'messages': [{'role': 'user',
    'content': [{'type': 'text',
      'text': '你是一名汽车保险承保专家。这里有一张车辆里程表的图片。请从中提取关键信息。'},
     {'type': 'image',
      'image': <PIL.Image.Image image mode=RGB size=426x320>}]},
   {'role': 'assistant',
    'content': [{'type': 'text',
      'text': '这张车辆里程表图片显示了以下关键信息：\n\n1. **总里程**：528,915公里\n2. **当前速度**：0公里/小时\n3. **当前时间**：19:18\n4. **当前温度**：+4.5°C\n5. **当前挡位**：停车挡（P）'}]}]},
 {'messages': [{'role': 'user',
    'content': [{'type': 'text',
      'text': '你是一名汽车保险承保专家。这里有一张车辆里程表的图片。请从中提取关键信息。'},
     {'type': 'image',
      'image': <PIL.Image.Image image mode=RGB size=800x445>}]},
   {'role': 'assistant',
    'content': [{'type': 'text',
      'text': '从这张车辆里程表的图片中，可以提取以下关键信息：\n\n1. **当前速度**：56.6英里/小时。\n2. **总里程**：37756英里。\n3. **当前时间**：11:14。\n4. **当前温度**：10.5°C。\n5. **当前挡位**：P（停车挡）。'}]}]}]

In [4]:
# 查看第一个样本的结构
print("第一个训练样本结构：")
print(f"用户消息文本: {converted_dataset[0]['messages'][0]['content'][0]['text']}")
print(f"助手回复: {converted_dataset[0]['messages'][1]['content'][0]['text']}")

# 训练前推理测试
print("\n训练前模型推理测试...")
FastVisionModel.for_inference(model)  # 切换到推理模式

# 加载测试图片
test_image = Image.open("images/1-vehicle-odometer-reading.jpg").convert('RGB')
test_instruction = "你是一名汽车保险承保专家。这里有一张车辆里程表的图片。请从中提取关键信息。"

# 构建推理输入
messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": test_instruction}
    ]}
]

input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    test_image,
    input_text, 
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")

# 生成回复
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
print("训练前模型输出:")
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128,
                   use_cache=True, temperature=1.5, min_p=0.1)

第一个训练样本结构：
用户消息文本: 你是一名汽车保险承保专家。这里有一张车辆里程表的图片。请从中提取关键信息。
助手回复: 这张车辆里程表图片显示了以下关键信息：

1. **总里程**：528,915公里
2. **当前速度**：0公里/小时
3. **当前时间**：19:18
4. **当前温度**：+4.5°C
5. **当前挡位**：停车挡（P）

训练前模型推理测试...
训练前模型输出:
从提供的图片中提取的关键信息如下：

1. **速度表**：
   - 当前速度显示为20公里/小时。

2. **里程数**：
   - 现在车辆的总行驶里程是529891.5公里。

3. **车况**：
   - 行驶距离：2600000.7公里。
   - 制动测试：19,181.3公里。

4. **油耗**：
   - 高速段燃油消耗为44升（平均）。
   - 城市


In [5]:
# 开始训练
print("\n开始训练模型...")
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

# 切换到训练模式
FastVisionModel.for_training(model)

# 配置训练器
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=UnslothVisionDataCollator(model, tokenizer),  # 必须使用视觉数据整理器
    train_dataset=converted_dataset,
    args=SFTConfig(
        per_device_train_batch_size=2,  # 每设备批次大小
        gradient_accumulation_steps=4,  # 梯度累积步数
        warmup_steps=5,  # 预热步数
        max_steps=30,  # 训练步数，可改为 num_train_epochs=1 进行完整训练
        learning_rate=2e-4,  # 学习率
        logging_steps=1,  # 日志记录步数
        optim="adamw_8bit",  # 使用8bit AdamW优化器
        weight_decay=0.01,  # 权重衰减
        lr_scheduler_type="linear",  # 线性学习率调度
        seed=3407,  # 随机种子
        output_dir="outputs",  # 输出目录
        report_to="none",  # 不使用wandb等记录工具
        
        # 视觉微调必需配置
        remove_unused_columns=False,
        dataset_text_field="",
        dataset_kwargs={"skip_prepare_dataset": True},
    ),
    max_seq_length=2048,  # 序列最大长度参数移到这里
)

# 显示显存使用情况
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. 最大显存 = {max_memory} GB.")
print(f"{start_gpu_memory} GB 显存已使用.")

# 执行训练
trainer_stats = trainer.train()

# 显示训练完成后的显存和时间统计
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

print(f"训练用时: {trainer_stats.metrics['train_runtime']} 秒")
print(f"训练用时: {round(trainer_stats.metrics['train_runtime']/60, 2)} 分钟")
print(f"峰值显存使用: {used_memory} GB")
print(f"LoRA训练显存使用: {used_memory_for_lora} GB")
print(f"显存使用率: {used_percentage}%")
print(f"LoRA显存使用率: {lora_percentage}%")

# 训练后推理测试
print("\n训练后模型推理测试...")
FastVisionModel.for_inference(model)  # 切换到推理模式


开始训练模型...
Unsloth: Model does not have a default image size - using 512
GPU = NVIDIA GeForce RTX 4090. 最大显存 = 23.65 GB.
3.867 GB 显存已使用.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2 | Num Epochs = 30 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,084,928/3,000,000,000 (1.37% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
1,2.389500
2,2.397800
3,2.386400
4,2.313700
5,2.056400
6,1.768400
7,1.464600
8,1.244400
9,1.078900
10,0.934300


Unsloth: Will smartly offload gradients to save VRAM!
训练用时: 50.2063 秒
训练用时: 0.84 分钟
峰值显存使用: 3.867 GB
LoRA训练显存使用: 0.0 GB
显存使用率: 16.351%
LoRA显存使用率: 0.0%

训练后模型推理测试...


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2_5_VLForConditionalGeneration(
      (visual): Qwen2_5_VisionTransformerPretrainedModel(
        (patch_embed): Qwen2_5_VisionPatchEmbed(
          (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
        (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
        (blocks): ModuleList(
          (0-31): 32 x Qwen2_5_VLVisionBlock(
            (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
            (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
            (attn): Qwen2_5_VLVisionSdpaAttention(
              (qkv): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1280, out_features=3840, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1280, out_features=16, bias=False)
                )
                (lora_B): ModuleD

In [6]:
# 使用相同的测试样本
inputs = tokenizer(
    test_image,
    input_text,
    add_special_tokens=False, 
    return_tensors="pt",
).to("cuda")

print("训练后模型输出:")
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128,
                   use_cache=True, temperature=1.5, min_p=0.1)

# 保存模型
print("\n保存LoRA适配器...")
model.save_pretrained("car_insurance_lora_model")  # 本地保存
tokenizer.save_pretrained("car_insurance_lora_model")

print("训练完成！模型已保存到 car_insurance_lora_model 目录")

训练后模型输出:
从这张车辆里程表的图片中，可以提取以下关键信息：

1. **总里程**：528,915公里。
2. **当前速度**：0公里/小时。
3. **当前时间**：19:18（晚）。
4. **当前温度**：+4.6°C。
5. **当前挡位**：停车挡。<|im_end|>

保存LoRA适配器...
训练完成！模型已保存到 car_insurance_lora_model 目录


In [ ]:
# from unsloth import FastVisionModel
# model, tokenizer = FastVisionModel.from_pretrained(
#     model_name="car_insurance_lora_model",
#     load_in_4bit=True,
# )
# FastVisionModel.for_inference(model)
